In [3]:
import jpeg_toolbox

In [34]:
img = jpeg_toolbox.load('./src/image/input.jpeg')
img2 = jpeg_toolbox.load('./src/image/output.jpeg')

In [35]:
img['coef_arrays'][0][0][8]

np.float64(99.0)

In [36]:
img2['coef_arrays'][0][0][8]

np.float64(99.0)

In [93]:
img['coef_arrays'][1]

array([[-17.,   0.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], shape=(1000, 1504))

In [94]:
img2['coef_arrays'][1]

array([[-17.,   0.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], shape=(1000, 1504))

In [95]:
img['coef_arrays'][2]

array([[13.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], shape=(1000, 1504))

In [96]:
img2['coef_arrays'][2]

array([[13.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], shape=(1000, 1504))

In [4]:
img["quant_tables"][0]


array([[ 6,  4,  4,  6,  9, 14, 18, 22],
       [ 4,  4,  5,  7,  9, 21, 22, 20],
       [ 5,  5,  6,  9, 14, 21, 25, 20],
       [ 5,  6,  8, 10, 18, 31, 29, 22],
       [ 6,  8, 13, 20, 24, 39, 37, 28],
       [ 9, 13, 20, 23, 29, 37, 41, 33],
       [18, 23, 28, 31, 37, 44, 43, 36],
       [26, 33, 34, 35, 40, 36, 37, 36]], dtype=int32)

In [5]:
img['coef_arrays'][0][0,8]

np.float64(99.0)